# Python - the missing bits
## Practical <span style="color: cyan">metaprogramming</span> applications

#### Michał Korzycki - https://www.youtube.com/@BitsofData/streams

> #### <span style="color: cyan">[Metaclasses] are deeper magic</span> than 99% of users should ever worry about. 
> #### If you wonder whether you need them, you don’t 
>
> ##### <div style="text-align: right">-- Tim Peters, Inventor of the timsort algorithm</span>

> #### I call it "the advanced beginner metaclass trap".
> ##### <div style="text-align: right">... random comment on the internet</span>

> #### 99% of the time actually you only need `subclass_init`
> #####  `__subclass_init__`  was added to Python 3.6 in 2015

> ##### <div style="text-align: right">... yet another random comment on the internet</span>

# Agenda

- ### Metaclasses refresher
- ### *Final* classes
- ### *"Real"* Singletons are not evil
- ### Java-like Annotations are also Meta
- ### Meta-models for SQLAlchemy
- ### General guidelines

# Metaclasses refresher

`object.__new__(cls[, ...])`
Called to create a new instance of `class cls`

`object.__init__(self[, ...])`
Called after the instance has been created (by __new__())

`__new__()` is intended mainly to allow subclasses of immutable types <...> to customize instance creation. It is also commonly overridden in custom metaclasses in order to customize class creation.

```python
from weakref import WeakKeyDictionary

class FlyweightMeta(type):
    def __new__(cls, name, bases, attrs):
        attrs['cache'] = WeakKeyDictionary()
        return super(FlyweightMeta, cls).__new__(cls, name, bases, attrs)

    def __call__(cls, *args, **kwargs):
        key = (*args, tuple(kwargs.items()))
        cache = getattr(cls, 'cache')
        return cache.setdefault(key, super(FlyweightMeta, cls).__call__(*args, **kwargs))
```

##### PEP 487 – Simpler customisation of class creation

`__init_subclass__` a hook that initializes all subclasses of a given class

`__set_name__` a hook that is called on all the attribute (descriptors) defined in the class,

In [2]:
import weakref

class WeakAttribute:
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]()

    def __set__(self, instance, value):
        instance.__dict__[self.name] = weakref.ref(value)

    # this is the new initializer:
    def __set_name__(self, owner, name):
        self.name = name

`__prepare__` is invoked as a function before the evaluation of the class body. 

# <span style="color: cyan">*Final*</span> &nbsp;classes

```python
class EnumMeta(type):
    """
    Metaclass for Enum
    """
    @classmethod
    def __prepare__(metacls, cls, bases, **kwds):
        ...
        first_enum = bases[-1]
        if first_enum._member_names_:
            raise TypeError("Cannot extend enumerations")
        ...
```

# <span style="color: cyan">*"Real"*</span>&nbsp; Singletons are not evil

### It is all about <span style="color: cyan">*subclassing*</span>

> ## Singletons aren't <span style="color: cyan">*polymorphic*</span>.
> ### <div style="text-align: right">Wiki Wiki Web (wiki.c2.com)</span>

---

> ## When you have singletons in your code, it makes it hard to <span style="color: cyan">test</span> and debug.
> ### <div style="text-align: right">David Litvak</span>


```python
class ParentSingletonMeta(type):
    def __call__(cls, *args, **kwargs):
        parent = cls.__bases__[-1]
        if parent not in _singleton_instances:
            instance = super().__call__(*args, **kwargs)
            _singleton_instances[parent] = instance
        return _singleton_instances[parent]

class ABCParentSingletonMeta(ParentSingletonMeta, abc.ABCMeta):
    pass


```

```python
class WebPushProvider(metaclass=ABCParentSingletonMeta):
    @classmethod
    def get_webpush_notification_data(cls, title, body, delivery_id):
...
...
...
    @classmethod
    @abstractmethod
    def web_push(cls, domain, email, data, db=None):
        pass
```

```python
class DummyWebPushProvider(WebPushProvider):
    data = None

    @classmethod
    def web_push(cls, domain, email, data, db=None):
        cls.data = domain, email, data

                
class StandardWebPushProvider(WebPushProvider):
    @classmethod
    def web_push(cls, domain, email, data, db=None):
        subscriptions = get_subscriptions(domain, email, db)
...
```

# Java-like annotations are also <span style="color: cyan">*Meta*</span>

### Use `__docs__` to carry information

```python
@router.get("/messages/{slug}", response_model=List[Message])
@authorize('CONFIRMED')
async def get_messages(slug: str, Authorize: AuthJWT = Depends(), 
                       db: Session = Depends(get_db)):
    return crud.messages(slug, access_token_from_authorize(slug, Authorize), db)
```

```python
def authorize(permission=None, refresh=False):
    def _authorize(_func):
        @wraps(_func)
        async def wrapper(*args, **kwargs):
            try:
                Authorize = kwargs.get('Authorize')          
...
...
...
        wrapper.__docs__ = "Authorized"
        return wrapper
```

```python
@pytest.mark.parametrize("router_fixture", [
    ('agent-panel', api.agent_panel.router, ['/crmquestions-hello']),
...
...
...
], indirect=True)
def test_secops(router_fixture):
    path, router, whitelist = router_fixture
...
...
```

```python
def test_secops(router_fixture):
    path, router, whitelist = router_fixture
...
    for route in routes:
        assert   ((route.path, route.methods, route.endpoint.__docs__) 
               == (route.path, route.methods, 'Authorized'))
        
```

# Meta-models 
## For SQLAlchemy

```python
class Variable(RelCampaignMixin, RelConsumerMixin, IdTimestampMixin, Base):
    name = db.Column(db.String(32), nullable=False)
    value = db.Column(db.String(32), nullable=False)
    value_type = db.Column(db.String(TYPE_LEN), db.ForeignKey('value_type.id'))
    __table_args__ = (
        db.UniqueConstraint('name', 'consumer_id', 'campaign_id',
                            name='uix_variable'),
    )
```

```python
class Event(EventPartitionMixin, Base, metaclass=PartitionByMeta, partition_by='created', partition_type='RANGE'):
    pass
```

```python
class EventPartitionMixin(PartitionMixin):
    slug = db.Column(db.String(32), nullable=False)
    consumer_id = db.Column(UUID(as_uuid=True), nullable=False)
    type = db.Column(EventTypeWrapper, nullable=False)
    payload = db.Column(JSON(), nullable=False, default={})
```

```python
class Message(MessagePartitionMixin, Base, metaclass=PartitionByMeta, 
              partition_by='created', partition_type='RANGE'):
    __table_args__ = (
        db.Index('message__email_slug_idx', 'email', 'slug'),
    )
```

```python
>>> class QuestBase:
...    # this is implicitly a @classmethod (see below for motivation)
...    def __init_subclass__(cls, swallow, **kwargs):
...        cls.swallow = swallow
...        super().__init_subclass__(**kwargs)

>>> class Quest(QuestBase, swallow="african"):
...    pass

>>> Quest.swallow
'african'
```

# General guidelines

- Be cautious about using Metaclasses
    - 99% of the time you just need `__init_subclass__`
- Do not mix metaclasses - that is not what the creators of libraries intended
- Read your Python Enhancement Proposals
    - *PEP 487 – Simpler customisation of class creation*


# <span style="color: cyan">Thank You!</span>


#### https://github.com/Bits-of-Data-PL/Metaprogramming-PyCon-2023
#### Like and subscribe: https://www.youtube.com/@BitsofData/streams


https://github.com/Bits-of-Data-PL/Metaprogramming-PyCon-2023 | https://www.youtube.com/@BitsofData/streams
:- | -:
!["qr.png"](qr.png) | !["qr2.png"](qr2.png)